In [3]:
!pip install pennylane

## I. PREPROCESAMIENTO DE LOS DATOS DE ENTRADA

In [4]:
import zipfile # Necesaria para la extracción del archivo .zip que contiene las carpetas con los conjuntos de datos
import pandas as pd # Útil para representación de métricas

with zipfile.ZipFile("titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_data")

df = pd.read_csv("titanic_data/train.csv")
df['Pclass'] = df['Pclass'].astype(str)
df = pd.concat([df, pd.get_dummies(df[['Pclass', 'Sex', 'Embarked']])], axis=1)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['is_child'] = df['Age'].map(lambda x: 1 if x < 12 else 0)

cols = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']
X = df[cols].astype(int).values
Y = (df['Survived'] * 2 - 1).values  # {0,1} → {-1,1}

## II. DIVISIÓN DE LOS DATOS DE ENTRENAMIENTO Y DATOS DE PRUEBA

In [5]:
from sklearn.model_selection import train_test_split
from pennylane import numpy as np

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train = np.array(X_train, requires_grad=False, dtype=int)
X_test = np.array(X_test, requires_grad=False, dtype=int)
Y_train = np.array(Y_train, requires_grad=False)
Y_test = np.array(Y_test, requires_grad=False)

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


## III. ETAPA DEL FEATURE MAP

In [6]:
def stateprep(x):
    qml.BasisEmbedding(x, wires=range(4))  # num_qubits = 4

## IV. ETAPA DEL ANSATZ

In [7]:
import pennylane as qml
num_qubits = 4
num_layers = 4
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev, interface="autograd")
def qnode_cost(param_pack):
    weights, x = param_pack
    stateprep(x)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, interface="autograd")
def qnode_only_weights(weights):
    stateprep(global_x_batch[0])
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

def classifier(weights, bias, x):
    return qnode_cost((weights, x)) + bias

## V. ETAPA DE FUNCIÓN DE PERDIDA

In [8]:
# La pérdida utilizada es: L = (y - pred)^2
# Se implementa de forma manual más adelante en la parte de entrenamiento

## VI. ETAPA DE EVALUACIÓN DE LA FUNCIÓN DE COSTO

In [9]:
from pennylane.optimize import QNGOptimizer
qng_opt = QNGOptimizer(stepsize=0.1)
bias_lr = 0.01

## VII. ETAPA DE ANÁLISIS DE PRECISIÓN

In [10]:
from sklearn.metrics import accuracy_score

def evaluate_accuracy(X, Y, weights, bias):
    preds = [np.sign(classifier(weights, bias, x)) for x in X]
    return accuracy_score(Y, preds)

## VIII. PESOS INICIALES DEFINIDOS

In [11]:
shape = qml.StronglyEntanglingLayers.shape(num_layers, num_qubits)
weights = 0.1 * np.random.randn(*shape)
weights.requires_grad = True
bias = np.tensor(0.0, requires_grad=True)

## IX. PESOS FINALES ENTRENADOS

In [12]:
print("== INICIO DEL ENTRENAMIENTO ==")
batch_size = 16
num_epochs = 100
log_interval = 5

acc_hist = []
bias_hist = []

for it in range(num_epochs):
    idx = np.random.choice(len(X_train), batch_size, replace=False)
    global_x_batch = X_train[idx]
    y_batch = Y_train[idx]

    # Paso 1: actualizar pesos
    metric_tensor = qml.adjoint_metric_tensor(qnode_only_weights)
    weights = qng_opt.step(qnode_only_weights, weights, metric_tensor_fn=metric_tensor)
    weights.requires_grad = True

    # Paso 2: actualizar bias
    preds = np.array([classifier(weights, bias, xi) for xi in global_x_batch])
    grads = -2 * (y_batch - preds)
    avg_grad = np.mean(grads)
    bias = bias - bias_lr * avg_grad

    if it % log_interval == 0 or it == num_epochs - 1:
        acc = evaluate_accuracy(X_train, Y_train, weights, bias)
        acc_hist.append(acc)
        bias_hist.append(bias.item())
        print(f"Iter {it:3d} | Accuracy: {acc:.3f} | Bias: {bias:.3f}")

== INICIO DEL ENTRENAMIENTO ==
Iter   0 | Accuracy: 0.520 | Bias: 0.002
Iter   5 | Accuracy: 0.508 | Bias: -0.043
Iter  10 | Accuracy: 0.517 | Bias: -0.063
Iter  15 | Accuracy: 0.618 | Bias: -0.088
Iter  20 | Accuracy: 0.605 | Bias: -0.087
Iter  25 | Accuracy: 0.624 | Bias: -0.102
Iter  30 | Accuracy: 0.338 | Bias: -0.104
Iter  35 | Accuracy: 0.688 | Bias: -0.129
Iter  40 | Accuracy: 0.683 | Bias: -0.115
Iter  45 | Accuracy: 0.584 | Bias: -0.079
Iter  50 | Accuracy: 0.584 | Bias: -0.036
Iter  55 | Accuracy: 0.633 | Bias: -0.000
Iter  60 | Accuracy: 0.633 | Bias: 0.062
Iter  65 | Accuracy: 0.633 | Bias: 0.107
Iter  70 | Accuracy: 0.633 | Bias: 0.157
Iter  75 | Accuracy: 0.633 | Bias: 0.201
Iter  80 | Accuracy: 0.633 | Bias: 0.244
Iter  85 | Accuracy: 0.633 | Bias: 0.278
Iter  90 | Accuracy: 0.633 | Bias: 0.314
Iter  95 | Accuracy: 0.633 | Bias: 0.329
Iter  99 | Accuracy: 0.633 | Bias: 0.333


## X. EVALUACIÓN DE DESEMPEÑO DEL CLASIFICADOR CUÁNTICO FINAL

In [13]:
print("\n== EVALUACIÓN FINAL ==")
acc_test = evaluate_accuracy(X_test, Y_test, weights, bias)
print(f"Test accuracy: {acc_test:.3f}")


== EVALUACIÓN FINAL ==
Test accuracy: 0.603
